# Practical Steps with FuzzManager

In this chapter we will perform basic steps around FuzzManager, including crash submission and triage as well as coverage measurement tasks.

**Prerequisites**

* This chapter builds directly upon the contents of the [last chapter](Fuzzing_in_the_Large.ipynb), so we recommend you read that before proceeding.
* This chapter requires you to be reading the book using the **Docker image**, as it uses a pre-installed demo instance of FuzzManager. Starting the Docker image is really straightforward...
* C/C++ knowledge is helpful if you would like to understand the crash examples. It is not required though to do so and you can perfectly learn the concepts of this chapter without fully understanding why these programs behave the way they do.

## Basic Crash Processing

To get started with basic steps in crash processing, let's take a look at *simply-buggy*, an example repository containing trivial C++ programs for illustration purposes.

In [1]:
!git clone https://github.com/choller/simply-buggy
!(cd simply-buggy && make)

Cloning into 'simply-buggy'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 13 (delta 4), reused 8 (delta 3), pack-reused 0
Unpacking objects: 100% (13/13), done.
clang++ -fsanitize=address -g -o simple-crash simple-crash.cpp
clang++ -fsanitize=address -g -o out-of-bounds out-of-bounds.cpp


The make command compiles our target program, including our first target, the *simple-crash* example. Alongside the program, there is also a configuration file generated. Let's take a look at that file and the source code:

In [2]:
!cat simply-buggy/simple-crash.cpp

/*
 * simple-crash - A simple NULL crash.
 *
 * WARNING: This program neither makes sense nor should you code like it is
 *          done in this program. It is purely for demo purposes and uses
 *          bad and meaningless coding habits on purpose.
 */

int crash() {
  int* p = (int*)0x1;
  *p = 0xDEADBEEF;
  return *p;
}

int main(int argc, char** argv) {
  return crash();
}


In [3]:
!cat simply-buggy/simple-crash.fuzzmanagerconf

[Main]
platform = x86-64
product = simple-crash-simple-crash
product_version = 60fc0b7d366288a8b0362a0c81ed3e2df13d6d3c
os = linux

[Metadata]
pathPrefix = /home/decoder/Mozilla/repos/fuzzingbook/notebooks/simply-buggy/
buildFlags = -fsanitize=address -g


As you can see, the source code is fairly simple: A forced crash by writing to a (near)-NULL pointer. The configuration file generated for the the binary also contains some straightforward information, like the revision of the program and other metadata that is required or at least useful later on when submitting things.

Running the program shows us a crash trace as expected:

In [6]:
!simply-buggy/simple-crash

AddressSanitizer:DEADLYSIGNAL
==25633==ERROR: AddressSanitizer: SEGV on unknown address 0x000000000001 (pc 0x000000516c4d bp 0x7ffd120e1700 sp 0x7ffd120e16d0 T0)
==25633==The signal is caused by a WRITE memory access.
==25633==Hint: address points to the zero page.
    #0 0x516c4c in crash() /home/decoder/Mozilla/repos/fuzzingbook/notebooks/simply-buggy/simple-crash.cpp:11:6
    #1 0x516cca in main /home/decoder/Mozilla/repos/fuzzingbook/notebooks/simply-buggy/simple-crash.cpp:16:10
    #2 0x7f5e0c178b96 in __libc_start_main /build/glibc-OTsEL5/glibc-2.27/csu/../csu/libc-start.c:310
    #3 0x41a399 in _start (/home/decoder/Mozilla/repos/fuzzingbook/notebooks/simply-buggy/simple-crash+0x41a399)

AddressSanitizer can not provide additional info.
SUMMARY: AddressSanitizer: SEGV /home/decoder/Mozilla/repos/fuzzingbook/notebooks/simply-buggy/simple-crash.cpp:11:6 in crash()
==25633==ABORTING


Now, what we would actually like to do is to run this binary from Python instead, detect that it crashed, collect the trace and submit it to the server. Let's start with a simple script that would just run the program we give it and detect the presence of the ASan trace:

In [14]:
import subprocess

cmd = ["simply-buggy/simple-crash"]

result = subprocess.run(cmd, stderr=subprocess.PIPE)
stderr = result.stderr.decode().splitlines()
crashed = False

for line in stderr:
    if "ERROR: AddressSanitizer" in line:
        crashed = True
        break

if crashed:
    print("Yay, we crashed!")
else:
    print("Move along, nothing to see...")

Yay, we crashed!


Nice, we can now run the binary and detect that it crashed. But how do we send this information to the server now? Let's make a few modifications...

In [23]:
import subprocess

from Collector.Collector import Collector
from FTB.ProgramConfiguration import ProgramConfiguration
from FTB.Signatures.CrashInfo import CrashInfo

# Instantiate the collector instance, this will be our entry point
# for talking to the server.
collector = Collector()

cmd = ["simply-buggy/simple-crash"]

result = subprocess.run(cmd, stderr=subprocess.PIPE)
stderr = result.stderr.decode().splitlines()
crashed = False

for line in stderr:
    if "ERROR: AddressSanitizer" in line:
        crashed = True
        break

if crashed:
    print("Yay, we crashed, processing...")
    
    # This reads the simple-crash.fuzzmanagerconf file
    configuration = ProgramConfiguration.fromBinary(cmd[0])
    
    # This reads and parses our ASan trace into a more generic format,
    # returning us a generic "CrashInfo" object that we can inspect
    # and/or submit to the server.
    crashInfo = CrashInfo.fromRawCrashData([], stderr, configuration)
    
    # Submit the crash
    collector.submit(crashInfo)
    
    print("Crash submitted!")
else:
    print("Move along, nothing to see...")

Yay, we crashed, processing...
Crash submitted!


We now submitted something to our local FuzzManager demo instance. If you go to https://127.0.0.1:8000/crashmanager/crashes/ you should see your crash. Now click on the crash to inspect the submitted data. Then click the orange *Create* button to create a bucket for this crash. A *crash signature* will be proposed to you for matching this and future crashes of the same type, accept it by clicking *Save*. You will be redirected to the newly created bucket, which shows you the size (how many crashes it holds), it's bug report status (buckets can be linked to bugs in an external bug tracker like Bugzilla) and many other useful information. If you click on the *Signatures* entry in the top menu, you should also see your newly created entry.

Buckets and their signatures are a central concept in FuzzManager. If you receive a lot of crash reports from various sources, bucketing allows you to easily group crashes and filter duplicates. The flexible signature system starts out with an initially proposed fine-grained signature, but it can be adjusted as needed to capture variations of the same bug and make tracking easier. In the next example, we will look at a more complex example that reads data from a file and creates multiple crash signatures.

In [26]:
!cat simply-buggy/out-of-bounds.cpp

/*
 * out-of-bounds - A simple multi-signature out-of-bounds demo.
 *
 * WARNING: This program neither makes sense nor should you code like it is
 *          done in this program. It is purely for demo purposes and uses
 *          bad and meaningless coding habits on purpose.
 */
#include <cstring>
#include <fstream>
#include <iostream>

void printFirst(char* data, size_t count) {
  std::string first(data, count);
  std::cout << first << std::endl;
}

void printLast(char* data, size_t count) {
  std::string last(data + strlen(data) - count, count);
  std::cout << last << std::endl;
}

int validateAndPerformAction(char* buffer, size_t size) {
  if (size < 2) {
    std::cerr << "Buffer is too short." << std::endl;
    return 1;
  }

  uint8_t action = buffer[0];
  uint8_t count = buffer[1];
  char* data = buffer + 2;

  if (!count) {
    std::cerr << "count must be non-zero." << std::endl;
    return 1;
  }

  // Forgot to check count vs. the length of data here, doh!

  if (!action) {


This program looks way more elaborate compared to the last one, but don't worry, it isn't really doing a whole lot. The code in the `main` function simply reads a file provided on the command line and puts its contents into a buffer that is passed to `validateAndPerformAction`. That function pulls out two bytes of the buffer (`action` and `count`) and considers the rest `data`. Depending on the value of `action`, it then calls either `printFirst` or `printLast`, which prints either the first or the last `count` bytes of `data`. Sounds pointless, and yes, it is. The whole idea of this program is that the security check (that `count` is not larger than the length of `data`) is missing in `validateAndPerformAction` but that the illegal access happens later in either of the two print functions. Hence, we would expect this program to generate at least two (slightly) different crash signatures. Let's try it out with very simple fuzzing based on the last Python script:

In [34]:
import random
import subprocess
import tempfile

from Collector.Collector import Collector
from FTB.ProgramConfiguration import ProgramConfiguration
from FTB.Signatures.CrashInfo import CrashInfo

# Instantiate the collector instance, this will be our entry point
# for talking to the server.
collector = Collector()

cmd = ["simply-buggy/out-of-bounds"]

crash_count = 0

for itnum in range(0,1000):
    rand_len = random.randint(1, 1024)
    rand_data = bytearray(os.urandom(rand_len))
    
    (fd, current_file) = tempfile.mkstemp(prefix="fuzztest")
    os.write(fd, rand_data)
    os.close(fd)
    
    current_cmd = []
    current_cmd.extend(cmd)
    current_cmd.append(current_file)
    
    result = subprocess.run(current_cmd, stderr=subprocess.PIPE)
    stderr = result.stderr.decode().splitlines()
    crashed = False

    for line in stderr:
        if "ERROR: AddressSanitizer" in line:
            crashed = True
            break

    if crashed:
        # This reads the simple-crash.fuzzmanagerconf file
        configuration = ProgramConfiguration.fromBinary(cmd[0])

        # This reads and parses our ASan trace into a more generic format,
        # returning us a generic "CrashInfo" object that we can inspect
        # and/or submit to the server.
        crashInfo = CrashInfo.fromRawCrashData([], stderr, configuration)

        # Submit the crash
        collector.submit(crashInfo, testCase = current_file)
        
        crash_count += 1
    
    os.remove(current_file)

print("Done, submitted %s crashes." % crash_count)

Done, submitted 3 crashes.
